<a href="https://colab.research.google.com/github/YasinnSoylu/portf-y-optimizasyonu/blob/main/portf%C3%B6y_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Varyans-Kovaryans Yöntemi ile risk portföy ve optimizasyon

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


import itertools

In [3]:
# Portföyde yer alacak hisse senetlerini belirleyin
tickers = ["AAPL", "GOOG", "TSLA", "MSFT","AMZN"]

# Verileri indirin
prices_df = yf.download(tickers, start="2023-01-01", end="2023-06-01", interval='1h')["Adj Close"]

prices_df

[*********************100%***********************]  5 of 5 completed


,AAPL,AMZN,GOOG,MSFT,TSLA
Datetime,,,,,
2023-01-03 09:30:00-05:00,125.459999,84.870399,89.485001,239.788101,111.059998
2023-01-03 10:30:00-05:00,125.345001,85.008400,89.629997,238.360001,108.470100
2023-01-03 11:30:00-05:00,124.660004,84.410004,89.205002,238.150101,106.150002
2023-01-03 12:30:00-05:00,124.649902,84.919998,89.389999,238.440002,105.669998
2023-01-03 13:30:00-05:00,124.570000,85.445000,89.779999,238.550003,107.320000
...,...,...,...,...,...
2023-05-31 11:30:00-04:00,178.742706,119.180099,123.769997,328.480011,196.386597
2023-05-31 12:30:00-04:00,178.889999,120.040001,124.040001,330.160004,198.029999
2023-05-31 13:30:00-04:00,179.169907,120.260002,123.885002,330.799988,200.500000


In [4]:
# Hisse senetleri getirilerini hesaplayın ve boş değerleri silin
returns_df = np.log(prices_df / prices_df.shift(1))
returns_df = returns_df.dropna()
returns_df



,AAPL,AMZN,GOOG,MSFT,TSLA
Datetime,,,,,
2023-01-03 10:30:00-05:00,-0.000917,0.001625,0.001619,-0.005973,-0.023596
2023-01-03 11:30:00-05:00,-0.005480,-0.007064,-0.004753,-0.000881,-0.021621
2023-01-03 12:30:00-05:00,-0.000081,0.006024,0.002072,0.001217,-0.004532
2023-01-03 13:30:00-05:00,-0.000641,0.006163,0.004353,0.000461,0.015494
2023-01-03 14:30:00-05:00,0.000401,0.008751,0.002892,0.003766,0.007103
...,...,...,...,...,...
2023-05-31 11:30:00-04:00,0.002368,-0.002471,0.000560,-0.003313,0.001996
2023-05-31 12:30:00-04:00,0.000824,0.007189,0.002179,0.005101,0.008333
2023-05-31 13:30:00-04:00,0.001563,0.001831,-0.001250,0.001937,0.012396


In [5]:
# Kovaryans matrisini hesaplayın
cov_matrix = returns_df.cov()
cov_matrix


,AAPL,AMZN,GOOG,MSFT,TSLA
AAPL,0.000033,0.000026,0.000023,0.000021,0.000042
AMZN,0.000026,0.000077,0.000041,0.000037,0.000054
GOOG,0.000023,0.000041,0.000063,0.000027,0.000033
MSFT,0.000021,0.000037,0.000027,0.000049,0.000032
TSLA,0.000042,0.000054,0.000033,0.000032,0.000188


In [6]:
# Portföy ağırlıklarını belirleyin (örneğin, 25% AAPL, 25% GOOG, 25% MSFT ve 25% AMZN)
weights = np.array([0.2, 0.2, 0.2, 0.2,0.2])

# Portföy getirisini hesaplayın
portfolio_return = np.sum(returns_df.mean() * weights) * 252

# Portföy getirisini yüzde olarak yazdırın
print("Portföy getirisi: {:.2f}%".format(portfolio_return * 100))

Portföy getirisi: 13.61%


In [7]:
# Portföy riskini hesaplayın
portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
portfolio_risk
# Portföy riskini yüzde olarak yazdırın
print("Portföy riski: {:.2f}%".format(portfolio_risk * 100))

Portföy riski: 10.45%


In [8]:
# Sharpe oranını hesaplayın
risk_free_rate = 0.1
sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_risk
print("Sharpe oranı: {:.2f}".format(sharpe_ratio))


Sharpe oranı: 0.35


In [9]:
# Minimum varyanslı portföyü bulun
num_assets = len(tickers)
ones = np.ones(num_assets)
cov_inv = np.linalg.inv(cov_matrix)
ones_cov_inv = ones.dot(cov_inv)
min_var_weights = ones_cov_inv / ones_cov_inv.sum()
min_var_weights


array([ 0.702609  , -0.01867486,  0.12920499,  0.25519772, -0.06833684])

In [10]:
# Minimum varyanslı portföyün ağırlıklarını yazdırın
print("Minimum varyanslı portföy ağırlıkları:")
for i in range(num_assets):
    ticker = tickers[i]
    weight = min_var_weights[i]
    print("{}: {:.2f}%".format(ticker, weight * 100))

Minimum varyanslı portföy ağırlıkları:
AAPL: 70.26%
GOOG: -1.87%
TSLA: 12.92%
MSFT: 25.52%
AMZN: -6.83%


In [11]:
#Minimum varyanslı portföyün ağırlıklarını normalize edin
min_var_weights /= np.sum(min_var_weights)

In [12]:
# Minimum varyanslı portföyün getirisini ve riskini hesaplayın
min_var_portfolio_return = np.sum(returns_df.mean() * min_var_weights) * 252
# Minimum varyanslı portföyün getirisini yüzde olarak yazdırın
print("Minimum varyanslı portföy getirisi: {:.2f}%".format(min_var_portfolio_return * 100))

Minimum varyanslı portföy getirisi: 11.11%


In [13]:
min_var_portfolio_risk = np.sqrt(np.dot(min_var_weights.T, np.dot(cov_matrix, min_var_weights))) * np.sqrt(252)

# Minimum varyanslı portföyün riskini yüzde olarak yazdırın
print("Minimum varyanslı portföy riski: {:.2f}%".format(min_var_portfolio_risk * 100))

Minimum varyanslı portföy riski: 8.40%


In [14]:
# Portföy değerini belirleyin (örneğin, 100,000 TL)
portfolio_value = 100000

# Her hisse senedi için lot sayısını hesaplayın
lot_sizes = {}
for i, ticker in enumerate(tickers):
  lot_sizes[ticker] = int(portfolio_value * weights[i] / prices_df[ticker][-1])


for ticker, lot_size in lot_sizes.items():

  print("{} lot sayısı: {}".format(ticker, lot_size))

AAPL lot sayısı: 112
GOOG lot sayısı: 161
TSLA lot sayısı: 98
MSFT lot sayısı: 60
AMZN lot sayısı: 165


In [15]:
# Minimum varyanslı portföyün ağırlıklarını kullanarak her bir hisse senedi için kaç lot alıp satmamız gerektiğini bulun
num_assets = len(tickers)
portfolio_lot_sizes = np.round(min_var_weights * portfolio_value / prices_df.iloc[-1], 0)
print("Her hisse senedi için alınması gereken lot sayısı:")
for i in range(num_assets):
    ticker = tickers[i]
    lot_size = int(portfolio_lot_sizes[i])
    print("{}: {} lot".format(ticker, lot_size))

Her hisse senedi için alınması gereken lot sayısı:
AAPL: 396 lot
GOOG: -15 lot
TSLA: 105 lot
MSFT: 78 lot
AMZN: -34 lot


In [16]:
num_assets = len(tickers)
portfolio_lot_sizes = np.round(min_var_weights * portfolio_value / prices_df.iloc[-1], 0)
print("Her hisse senedi için alınması gereken lot sayısı:")
for i in range(num_assets):
    ticker = tickers[i]
    lot_size = int(portfolio_lot_sizes[i])
    if lot_size >= 0:
        print("{}: {} lot al".format(ticker, lot_size))
    else:
        print("{}: {} lot sat".format(ticker, abs(lot_size)))

Her hisse senedi için alınması gereken lot sayısı:
AAPL: 396 lot al
GOOG: 15 lot sat
TSLA: 105 lot al
MSFT: 78 lot al
AMZN: 34 lot sat


# lot based

In [17]:
# Hisseleri ve lot sayılarını belirle
tickers = ['AAPL', 'GOOG', 'TSLA', 'MSFT', 'AMZN']
lot_sizes = [10, 20, 30, 40, 50]

# Veri çekme işlemi
start_date = '2023-03-01'
end_date = '2023-06-01'

prices_df = pd.DataFrame()

for ticker in tickers:
    prices = yf.download(ticker, start=start_date, end=end_date,interval="1h")
    prices_df[ticker] = prices['Close']
prices_df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,AAPL,GOOG,TSLA,MSFT,AMZN
Datetime,,,,,
2023-03-01 09:30:00-05:00,145.669998,90.224998,200.139999,247.660004,92.940002
2023-03-01 10:30:00-05:00,145.949997,90.119301,199.880005,247.369995,92.400002
2023-03-01 11:30:00-05:00,146.419998,90.779999,201.946106,247.789902,92.815002
2023-03-01 12:30:00-05:00,145.970001,90.671204,200.998001,247.453293,92.209999
2023-03-01 13:30:00-05:00,145.360001,90.349998,201.879898,246.519897,91.669998
...,...,...,...,...,...
2023-05-31 11:30:00-04:00,178.742706,123.769997,196.386597,328.480011,119.180099
2023-05-31 12:30:00-04:00,178.889999,124.040001,198.029999,330.160004,120.040001
2023-05-31 13:30:00-04:00,179.169907,123.885002,200.500000,330.799988,120.260002


In [18]:
# Hisselerin son kapanış fiyatlarını al ve lot sayıları ile çarp
last_prices = prices_df.iloc[-1]
lot_prices = last_prices * lot_sizes
lot_prices


AAPL     1774.199982
GOOG     2471.199951
TSLA     6117.899780
MSFT    13148.000488
AMZN     6030.599976
Name: 2023-05-31 15:30:00-04:00, dtype: float64

In [19]:
# Portföy değerini hesapla
portfolio_value = lot_prices.sum()
portfolio_value

29541.900177001953

In [20]:
  # Hisselerin getirilerini hesapla
returns = prices_df.pct_change().dropna()
returns

,AAPL,GOOG,TSLA,MSFT,AMZN
Datetime,,,,,
2023-03-01 10:30:00-05:00,0.001922,-0.001171,-0.001299,-0.001171,-0.005810
2023-03-01 11:30:00-05:00,0.003220,0.007331,0.010337,0.001697,0.004491
2023-03-01 12:30:00-05:00,-0.003073,-0.001198,-0.004695,-0.001358,-0.006518
2023-03-01 13:30:00-05:00,-0.004179,-0.003543,0.004388,-0.003772,-0.005856
2023-03-01 14:30:00-05:00,-0.000929,-0.000775,0.002873,-0.000973,0.002673
...,...,...,...,...,...
2023-05-31 11:30:00-04:00,0.002370,0.000560,0.001998,-0.003308,-0.002468
2023-05-31 12:30:00-04:00,0.000824,0.002182,0.008368,0.005114,0.007215
2023-05-31 13:30:00-04:00,0.001565,-0.001250,0.012473,0.001938,0.001833


In [21]:
cov_matrix = returns.cov()
cov_matrix

,AAPL,GOOG,TSLA,MSFT,AMZN
AAPL,0.000028,0.000018,0.000028,0.000018,0.000022
GOOG,0.000018,0.000046,0.000023,0.000022,0.000027
TSLA,0.000028,0.000023,0.000133,0.000017,0.000040
MSFT,0.000018,0.000022,0.000017,0.000044,0.000029
AMZN,0.000022,0.000027,0.000040,0.000029,0.000060


In [22]:
# Negatif portföy getirisini minimize eden fonksiyon
def portfolio_variance(weights, cov_matrix):
    return np.dot(weights.T, np.dot(cov_matrix, weights))

def minimize_variance(weights, cov_matrix):
    return minimize(portfolio_variance, weights, args=(cov_matrix,), method='BFGS', options={'disp': False}).x

In [23]:
# Optimum portföyü bul ve al-sat kararlarını yazdır
weights = np.array([1/len(tickers)]*len(tickers))
optimized_weights = minimize_variance(weights, cov_matrix)
optimized_lot_sizes = np.floor(optimized_weights * portfolio_value / last_prices)
trade_sizes = optimized_lot_sizes - np.array(lot_sizes)
for i, ticker in enumerate(tickers):
    if trade_sizes[i] > 0:
        print(f"Satılacak {ticker} lot sayısı: {int(trade_sizes[i])}")
    elif trade_sizes[i] < 0:
        print(f"Alınacak {ticker} lot sayısı: {int(-trade_sizes[i])}")
    else:
        print(f"{ticker} için herhangi bir işlem yapılması gerekmiyor.")

AAPL için herhangi bir işlem yapılması gerekmiyor.
Alınacak GOOG lot sayısı: 12
Alınacak TSLA lot sayısı: 39
Alınacak MSFT lot sayısı: 37
Alınacak AMZN lot sayısı: 52


#optimum portföy oluşturma

In [36]:
# hisse senetlerini belirle
tickers = ['AAPL', 'GOOG', 'MSFT', 'AMZN', 'TSLA', 'BRK-A', 'V', 'JNJ', 'JPM',
           'UNH', 'HD', 'PG', 'MA', 'DIS', 'NVDA', 'VZ', 'ADBE', 'BAC', 'XOM']

# verileri yfinance'den yükle
data = yf.download(tickers, start="2023-01-01", end="2023-06-01", interval='1h')
data

[*********************100%***********************]  19 of 19 completed


Adj Close                                     \
                                 AAPL        ADBE        AMZN        BAC   
Datetime                                                                   
2023-01-03 09:30:00-05:00  125.459999  336.670013   84.870399  33.519299   
2023-01-03 10:30:00-05:00  125.345001  335.000000   85.008400  33.770000   
2023-01-03 11:30:00-05:00  124.660004  333.372498   84.410004  33.525002   
2023-01-03 12:30:00-05:00  124.649902  335.015015   84.919998  33.415001   
2023-01-03 13:30:00-05:00  124.570000  335.904999   85.445000  33.345001   
...                               ...         ...         ...        ...   
2023-05-31 11:30:00-04:00  178.742706  415.214996  119.180099  27.559999   
2023-05-31 12:30:00-04:00  178.889999  419.165009  120.040001  27.594999   
2023-05-31 13:30:00-04:00  179.169907  419.239990  120.260002  27.733299   
2023-05-31 14:30:00-04:00  178.717896  417.480011  120.055000  27.795000   
2023-05-31 15:30:00-04:00  177.419998  417.950012  120.612000  27.799999   

                                                                            \
                                  BRK-A        DIS        GOOG          HD   
Datetime                                                                     
2023-01-03 09:30:00-05:00  469249.62500  88.348000   89.485001  314.445007   
2023-01-03 10:30:00-05:00  467428.43750  88.489998   89.629997  314.179901   
2023-01-03 11:30:00-05:00  466306.37500  88.067200   89.205002  313.440002   
2023-01-03 12:30:00-05:00  466547.40625  88.199997   89.389999  316.024994   
2023-01-03 13:30:00-05:00  467062.75000  88.419998   89.779999  315.195007   
...                                 ...        ...         ...         ...   
2023-05-31 11:30:00-04:00  487466.00000  87.339996  123.769997  283.149902   
2023-05-31 12:30:00-04:00  486704.46875  87.680000  124.040001  284.190002   
2023-05-31 13:30:00-04:00  487467.00000  87.968597  123.885002  286.260010   
2023-05-31 14:30:00-04:00  488719.96875  88.010002  124.040001  285.440002   
2023-05-31 15:30:00-04:00  487934.00000  87.980003  123.559998  283.480011   

                                                   ...   Volume           \
                                  JNJ         JPM  ...      JPM       MA   
Datetime                                           ...                     
2023-01-03 09:30:00-05:00  177.289993  135.110001  ...  1905951  1256296   
2023-01-03 10:30:00-05:00  176.910004  135.533005  ...  1241892   252627   
2023-01-03 11:30:00-05:00  177.470001  134.529999  ...   743374   190102   
2023-01-03 12:30:00-05:00  177.380005  134.300003  ...  1007933   154119   
2023-01-03 13:30:00-05:00  177.240005  134.225006  ...   624752   118496   
...                               ...         ...  ...      ...      ...   
2023-05-31 11:30:00-04:00  154.630005  134.774994  ...   723659   335033   
2023-05-31 12:30:00-04:00  154.330002  134.866699  ...   704281   319913   
2023-05-31 13:30:00-04:00  154.679993  135.119995  ...  1000727   335625   
2023-05-31 14:30:00-04:00  155.274994  135.600006  ...  1272745   386146   
2023-05-31 15:30:00-04:00  155.160004  135.740005  ...  1484055   968949   

                                                                         \
                              MSFT      NVDA       PG      TSLA     UNH   
Datetime                                                                  
2023-01-03 09:30:00-05:00  6337273  11605646  1416073  65615599  610033   
2023-01-03 10:30:00-05:00  3058832   7234854   767988  33929104  421172   
2023-01-03 11:30:00-05:00  2236397   4647010   617339  33451392  419235   
2023-01-03 12:30:00-05:00  2596124   3367135   433372  24087601  303792   
2023-01-03 13:30:00-05:00  1635805   3274138   405138  23360158  254528   
...                            ...       ...      ...       ...     ...   
2023-05-31 11:30:00-04:00  3164836  11025970   614288  13592671  252672   
2023-05-31 12:30:00-04:00  2432630   9423186   574156  

In [37]:
daily_returns = data['Adj Close'].pct_change()
std_devs = daily_returns.std()

daily_returns


,AAPL,ADBE,AMZN,BAC,BRK-A,DIS,GOOG,HD,JNJ,JPM,MA,MSFT,NVDA,PG,TSLA,UNH,V,VZ,XOM
Datetime,,,,,,,,,,,,,,,,,,,
2023-01-03 09:30:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-03 10:30:00-05:00,-0.000917,-0.004960,0.001626,0.007479,-0.003881,0.001607,0.001620,-0.000843,-0.002143,0.003131,-0.004514,-0.005956,-0.010870,-0.003064,-0.023320,-0.008942,-0.005694,-0.001892,-0.007092
2023-01-03 11:30:00-05:00,-0.005465,-0.004858,-0.007039,-0.007255,-0.002401,-0.004778,-0.004742,-0.002355,0.003165,-0.007400,-0.009299,-0.000881,-0.000915,0.000000,-0.021389,-0.000268,-0.004957,0.003411,-0.003807
2023-01-03 12:30:00-05:00,-0.000081,0.004927,0.006042,-0.003281,0.000517,0.001508,0.002074,0.008247,-0.000507,-0.001710,0.002347,0.001217,0.006768,0.004776,-0.004522,0.000890,0.001354,0.003777,-0.007726
2023-01-03 13:30:00-05:00,-0.000641,0.002657,0.006182,-0.002095,0.001105,0.002494,0.004363,-0.002626,-0.000789,-0.000558,-0.001098,0.000461,-0.003630,0.000499,0.015615,-0.003276,-0.000628,0.004641,-0.006304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-31 11:30:00-04:00,0.002370,0.002644,-0.002468,0.001272,0.001679,0.001370,0.000560,0.001078,0.000323,0.001598,0.003598,-0.003308,-0.015606,0.001259,0.001998,0.003580,0.003035,0.004492,-0.001512
2023-05-31 12:30:00-04:00,0.000824,0.009513,0.007215,0.001270,-0.001562,0.003893,0.002182,0.003673,-0.001940,0.000680,0.002168,0.005114,0.009721,0.001606,0.008368,-0.000809,0.003025,-0.002935,-0.002150
2023-05-31 13:30:00-04:00,0.001565,0.000179,0.001833,0.005012,0.001567,0.003291,-0.001250,0.007284,0.002268,0.001878,0.004215,0.001938,0.000545,0.001674,0.012473,0.002034,0.006147,-0.002803,-0.000147


In [38]:
std_devs

AAPL     0.005745
ADBE     0.007777
AMZN     0.008795
BAC      0.007032
BRK-A    0.003977
DIS      0.006898
GOOG     0.007924
HD       0.005732
JNJ      0.003627
JPM      0.006114
MA       0.004941
MSFT     0.007060
NVDA     0.014743
PG       0.003991
TSLA     0.013765
UNH      0.005438
V        0.004581
VZ       0.004705
XOM      0.006799
dtype: float64

In [39]:
# Tüm 3'lü kombinasyonları oluşturma
combinations = itertools.combinations(tickers, 3)

# En iyi portföyü bulmak için kullanacağımız değişkenler
best_portfolio_returns = float('-inf')
best_portfolio_risk = float('inf')
best_portfolio = None

# En düşük riskli portföyü bulmak için kullanacağımız değişkenler
lowest_risk_portfolio_returns = None
lowest_risk_portfolio_risk = float('inf')
lowest_risk_portfolio = None


# En düşük riskli portföyü bulmak için kullanacağımız değişkenler
lowest_risk_portfolio_returns_post = None
lowest_risk_portfolio_risk_post = float('inf')
lowest_risk_portfolio_post = None

In [40]:
for combination in combinations:
    # Seçilen hisselerin günlük getirileri ve standart sapmalarını hesapla
    daily_returns_subset = daily_returns[list(combination)]
    std_dev_subset = std_devs[list(combination)]

    # Tüm olası ağırlıkların kombinasyonlarını oluşturma
    weights_combinations = itertools.product([0.1,0.25,0.50,0.75, 1], repeat=len(combination))

    for weights in weights_combinations:
        # Ağırlıkları normalize et
        weights = np.array(weights) / sum(weights)

        # Portföy getirisi ve riskini hesapla
        portfolio_daily_returns = (daily_returns_subset * weights).sum(axis=1)
        portfolio_std_dev = np.sqrt((weights ** 2 * std_dev_subset ** 2).sum())
        portfolio_annual_returns = (1 + portfolio_daily_returns.mean()) ** 252 - 1

        # En iyi portföyü bulma
        if portfolio_annual_returns > best_portfolio_returns:
            best_portfolio_returns = portfolio_annual_returns
            best_portfolio_risk = portfolio_std_dev
            best_portfolio = combination, weights

        # En düşük riskli portföyü bulma
        if portfolio_std_dev < lowest_risk_portfolio_risk:
            lowest_risk_portfolio_risk = portfolio_std_dev
            lowest_risk_portfolio_returns = portfolio_annual_returns
            lowest_risk_portfolio = combination, weights


        # En düşük riskli portföyü bulma
        if portfolio_std_dev < lowest_risk_portfolio_risk_post and portfolio_annual_returns >= 0:
            lowest_risk_portfolio_risk_post = portfolio_std_dev
            lowest_risk_portfolio_returns_post = portfolio_annual_returns
            lowest_risk_portfolio_post = combination, weights

In [41]:
# En iyi portföyü yazdırma
print("\nEn iyi portföy: ")
print(f"Hisse senetleri: {best_portfolio[0]}")
print("Ağırlıklar: " + " ".join([f"{weight:.2%}" for weight in best_portfolio[1]]))
print(f"Yıllık getiri: {best_portfolio_returns:.2%}")
print(f"Risk: {best_portfolio_risk:.2%}")

# En düşük riskli portföyü yazdırma
print("\nEn düşük riskli portföy: ")
print(f"Hisse senetleri: {lowest_risk_portfolio[0]}")
print("Ağırlıklar: " + " ".join([f"{weight:.2%}" for weight in lowest_risk_portfolio[1]]))
print(f"Yıllık getiri: {lowest_risk_portfolio_returns:.2%}")
print(f"Risk: {lowest_risk_portfolio_risk:.2%}")


#En düşük riskli pozitif getirili  portföyü yazdırma getiri negatif olmadan
print("\nEn düşük riskli portföy: ")
print(f"Hisse senetleri: {lowest_risk_portfolio_post[0]}")
print("Ağırlıklar: " + " ".join([f"{weight:.2%}" for weight in lowest_risk_portfolio_post[1]]))
print(f"Yıllık getiri: {lowest_risk_portfolio_returns_post:.2%}")
print(f"Risk: {lowest_risk_portfolio_risk_post:.2%}")


En iyi portföy: 
Hisse senetleri: ('AMZN', 'TSLA', 'NVDA')
Ağırlıklar: 8.33% 8.33% 83.33%
Yıllık getiri: 39.67%
Risk: 1.24%

En düşük riskli portföy: 
Hisse senetleri: ('BRK-A', 'JNJ', 'PG')
Ağırlıklar: 30.00% 40.00% 30.00%
Yıllık getiri: -1.79%
Risk: 0.22%

En düşük riskli portföy: 
Hisse senetleri: ('BRK-A', 'V', 'JNJ')
Ağırlıklar: 42.86% 28.57% 28.57%
Yıllık getiri: 0.02%
Risk: 0.24%
